In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)

In [3]:
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

In [4]:
model_outputs_dir = params['model_dir']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

In [5]:
# outputs of CT-RAMP model for tour and trip file
household_model_dir = _join(model_outputs_dir, "main")

# input household and person data
person_file = _join(ctramp_dir, 'main\\personData_' + str(iteration) + '.csv')
household_file = _join(ctramp_dir, 'main\\householdData_' + str(iteration) + '.csv')

person = pd.read_csv(person_file)

hh = pd.read_csv(household_file, usecols = ['hh_id', 'taz'])
hh = hh.rename(columns = {'taz': 'home_zone'})

#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(params['common_dir'], "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(params['common_dir'], "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

# transbay od pairs
transbay_od = pd.read_csv(_join(params['common_dir'], "transbay_od.csv")) #columns = transbay_o, transbay_d

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")

In [6]:
link21_purp_mapping = params['purpose_mapping']
link21_purp_mapping

{'work_very high': 'work',
 'shopping': 'shopping',
 'othdiscr': 'othdiscr',
 'school_grade': 'school',
 'work_high': 'work',
 'escort_kids': 'escort',
 'othmaint': 'othmaint',
 'work_med': 'work',
 'atwork_eat': 'social',
 'eatout': 'social',
 'work_low': 'work',
 'social': 'social',
 'university': 'school',
 'escort_no kids': 'escort',
 'school_high': 'school',
 'atwork_business': 'business',
 'atwork_maint': 'othmaint',
 'Home': 'home',
 'work': 'work',
 'Work': 'work',
 'atwork': 'work',
 'escort': 'escort',
 'school': 'school',
 'business': 'business'}

In [71]:
def create_trip_roster(ctramp_dir, transbay_od, geo_cwks, link21_purp_mapping):
    
    
    ind_trip = pd.read_csv(_join(ctramp_dir, 'main\\indivTripData_' + str(iteration) + '.csv'))
    jnt_trip = pd.read_csv(_join(ctramp_dir, 'main\\jointTripData_' + str(iteration) + '.csv'))
    
    jnt_trip['tours'] = 'joint'
    ind_trip['tours'] = 'inm'
    
    ind_drop_columns = ['avAvailable', 'sampleRate', 'taxiWait', 'singleTNCWait', 
                    'sharedTNCWait', 'orig_walk_segment', 'dest_walk_segment',
                    'person_id', 'person_num', 'parking_taz']

    jnt_drop_columns = ['avAvailable', 'sampleRate', 'taxiWait', 'singleTNCWait', 
                    'sharedTNCWait', 'orig_walk_segment', 'dest_walk_segment',
                'parking_taz', 'num_participants']

    ind_trip = ind_trip.drop(columns = ind_drop_columns)
    jnt_trip = jnt_trip.drop(columns = jnt_drop_columns)

    out_tripdata = pd.concat([ind_trip, jnt_trip])
    
    # add transbay_od to final tours
    out_tripdata = pd.merge(out_tripdata, transbay_od, left_on= ['orig_taz', 'dest_taz'], right_on = ['transbay_o', 'transbay_d'], how = 'left')
    out_tripdata['transbay_od'] = out_tripdata['transbay_od'].fillna(0)

    out_tripdata = out_tripdata.drop(columns = ['transbay_o', 'transbay_d'])
    #print(out_tripdata['transbay_od'].value_counts())

    # add geographies to final tours
    out_tripdata = pd.merge(out_tripdata, geo_cwks, left_on = ['orig_taz'], right_on = ['taz'], how = 'left')
    out_tripdata = out_tripdata.rename(columns = {'rdm_zones':'orig_rdm_zones', 
                                                'super_district': 'orig_super_dist',
                                                'county': 'orig_county'})
    del out_tripdata['taz']

    out_tripdata = pd.merge(out_tripdata, geo_cwks, left_on = ['dest_taz'], right_on = ['taz'], how = 'left')
    out_tripdata = out_tripdata.rename(columns = {'rdm_zones':'dest_rdm_zones', 
                                                'super_district': 'dest_super_dist',
                                                'county': 'dest_county'})

    del out_tripdata['taz']

    out_tripdata = pd.merge(out_tripdata, hh, on = 'hh_id', how = 'left')

    # add prioirty population
    out_tripdata = pd.merge(out_tripdata, pp_perc, left_on = ['home_zone'], right_on = ['taz'], how = 'left')
    print("NAs in PP Share:",  out_tripdata['pp_share'].isna().sum())
    # out_tourdata['pp_share'] = out_tourdata['pp_share'].fillna(0)
    del out_tripdata['taz']
    
    #add link21 purpose definitions
    df = out_tripdata.copy()
    df['new_dest_purp'] = df['dest_purpose']
    df['new_orig_purp'] = df['orig_purpose']
    
    # changing the purpose categories for atwork purpose
    df.loc[(df['tour_purpose'] == 'atwork_eat') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'eatout'
    df.loc[(df['tour_purpose'] == 'atwork_eat') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'eatout'

    df.loc[(df['tour_purpose'] == 'atwork_business') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'business'
    df.loc[(df['tour_purpose'] == 'atwork_business') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'business'

    df.loc[(df['tour_purpose'] == 'atwork_maint') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'othmaint'
    df.loc[(df['tour_purpose'] == 'atwork_maint') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'othmaint'
    
    # adding new link21 trip purpose
    df['link21_tour_purp'] = df['tour_purpose'].map(link21_purp_mapping)
    df['link21_orig_purp'] = df['new_orig_purp'].map(link21_purp_mapping)
    df['link21_dest_purp'] = df['new_dest_purp'].map(link21_purp_mapping)

    df['link21_trip_purp'] = df['link21_dest_purp']
    
    # for last trip on tour
    df1 = df.loc[(df['link21_dest_purp'] == 'home')]
    conditions = [
        df1['link21_tour_purp'].eq('work'),
        df1['link21_tour_purp'].eq('school'),
        ~df1['link21_tour_purp'].isin(['work','school'])
    ]

    choices = ['work', 'school', df1['link21_orig_purp']]
    df1['link21_trip_purp'] = np.select(conditions, choices, default=0)
    df2 = df.loc[(df['link21_dest_purp'] != 'home')]
    df2['link21_trip_purp'] = df2['link21_dest_purp']
    df = pd.concat([df1, df2], ignore_index=True)
    
    df1 = df.loc[df['dest_purpose'] == 'atwork']
    conditions = [
        df1['link21_tour_purp'].eq('business'),
        ~df1['link21_tour_purp'].eq('business')
    ]
    choices = ['business', df1['link21_orig_purp']]
    df1['link21_trip_purp'] = np.select(conditions, choices, default=0)
    
    df2 = df.loc[(df['dest_purpose'] != 'atwork')]
    df = pd.concat([df1, df2], ignore_index=True)
    
    df['trips'] = 1
    
    return df

In [7]:
# Read Data
# input trips
ind_trip = pd.read_csv(_join(ctramp_dir, 'main\\indivTripData_' + str(iteration) + '.csv'))
jnt_trip = pd.read_csv(_join(ctramp_dir, 'main\\jointTripData_' + str(iteration) + '.csv'))

# Checks
print("total joint trips:", len(jnt_trip))
print("total inm trips:", len(ind_trip))
print("Sample Rate:", jnt_trip['sampleRate'].unique()," ",ind_trip['sampleRate'].unique())

jnt_trip['tours'] = 'joint'
ind_trip['tours'] = 'inm'

ind_drop_columns = ['avAvailable', 'sampleRate', 'taxiWait', 'singleTNCWait', 
                    'sharedTNCWait', 'orig_walk_segment', 'dest_walk_segment',
                    'person_id', 'person_num', 'parking_taz']

jnt_drop_columns = ['avAvailable', 'sampleRate', 'taxiWait', 'singleTNCWait', 
                    'sharedTNCWait', 'orig_walk_segment', 'dest_walk_segment',
                'parking_taz', 'num_participants']

ind_trip = ind_trip.drop(columns = ind_drop_columns)
jnt_trip = jnt_trip.drop(columns = jnt_drop_columns)

out_tripdata = pd.concat([ind_trip, jnt_trip])

# add transbay_od to final tours
out_tripdata = pd.merge(out_tripdata, transbay_od, left_on= ['orig_taz', 'dest_taz'], right_on = ['transbay_o', 'transbay_d'], how = 'left')
out_tripdata['transbay_od'] = out_tripdata['transbay_od'].fillna(0)

out_tripdata = out_tripdata.drop(columns = ['transbay_o', 'transbay_d'])
#print(out_tripdata['transbay_od'].value_counts())

# add geographies to final tours
out_tripdata = pd.merge(out_tripdata, geo_cwks, left_on = ['orig_taz'], right_on = ['taz'], how = 'left')
out_tripdata = out_tripdata.rename(columns = {'rdm_zones':'orig_rdm_zones', 
                                            'super_district': 'orig_super_dist',
                                            'county': 'orig_county'})
del out_tripdata['taz']

out_tripdata = pd.merge(out_tripdata, geo_cwks, left_on = ['dest_taz'], right_on = ['taz'], how = 'left')
out_tripdata = out_tripdata.rename(columns = {'rdm_zones':'dest_rdm_zones', 
                                            'super_district': 'dest_super_dist',
                                            'county': 'dest_county'})

del out_tripdata['taz']

out_tripdata = pd.merge(out_tripdata, hh, on = 'hh_id', how = 'left')

# add prioirty population
out_tripdata = pd.merge(out_tripdata, pp_perc, left_on = ['home_zone'], right_on = ['taz'], how = 'left')
print("NAs in PP Share:",  out_tripdata['pp_share'].isna().sum())
# out_tourdata['pp_share'] = out_tourdata['pp_share'].fillna(0)
del out_tripdata['taz']

print("total trips:", len(out_tripdata))
print(out_tripdata['tours'].value_counts())

#Create non-transit trip data
out_tripdata_nontransit = out_tripdata[out_tripdata['trip_mode'].isin([1,2,3,4,5,9])]
print('total non transit trips:', len(out_tripdata_nontransit))

#Create transit only trip data
out_tripdata_transit = out_tripdata[out_tripdata['trip_mode'].isin([6,7,8])]
print('total transit trips:', len(out_tripdata_transit))

total joint trips: 80279
total inm trips: 3762667
Sample Rate: [0.15]   [0.15]
NAs in PP Share: 0
total trips: 3842946
inm      3762667
joint      80279
Name: tours, dtype: int64
total non transit trips: 3633844
total transit trips: 209102


In [8]:
ind_trip = pd.read_csv(_join(ctramp_dir, 'main\\indivTripData_' + str(iteration) + '.csv'))
jnt_trip = pd.read_csv(_join(ctramp_dir, 'main\\jointTripData_' + str(iteration) + '.csv'))

In [9]:
ind_trip.columns

Index(['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound',
       'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_taz',
       'orig_walk_segment', 'dest_taz', 'dest_walk_segment', 'parking_taz',
       'depart_hour', 'trip_mode', 'tour_mode', 'tour_category', 'avAvailable',
       'sampleRate', 'taxiWait', 'singleTNCWait', 'sharedTNCWait'],
      dtype='object')

In [11]:
jnt_trip.columns

Index(['hh_id', 'tour_id', 'stop_id', 'inbound', 'tour_purpose',
       'orig_purpose', 'dest_purpose', 'orig_taz', 'orig_walk_segment',
       'dest_taz', 'dest_walk_segment', 'parking_taz', 'depart_hour',
       'trip_mode', 'num_participants', 'tour_mode', 'tour_category',
       'avAvailable', 'sampleRate', 'taxiWait', 'singleTNCWait',
       'sharedTNCWait'],
      dtype='object')

In [11]:
out_tripdata[out_tripdata['trip_mode']==7]

,hh_id,tour_id,stop_id,inbound,tour_purpose,orig_purpose,dest_purpose,orig_taz,dest_taz,depart_hour,...,tours,transbay_od,orig_rdm_zones,orig_super_dist,orig_county,dest_rdm_zones,dest_super_dist,dest_county,home_zone,pp_share
125,1453060,0,-1,0,work_med,Home,work,1,2415,7,...,inm,1.0,75,26,4,105,14,4,1,0.0
126,1453060,0,-1,1,work_med,work,Home,2415,1,18,...,inm,1.0,105,14,4,75,26,4,1,0.0
18895,1290067,0,-1,0,work_high,Home,work,21,2291,5,...,inm,1.0,52,12,5,214,31,4,21,0.0
18896,1290067,0,-1,1,work_high,work,Home,2291,21,23,...,inm,1.0,214,31,4,52,12,5,21,0.0
24385,1294856,0,-1,0,work_low,Home,work,27,795,5,...,inm,0.0,24,23,9,139,10,2,27,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818378,674572,0,-1,1,shopping,shopping,Home,2728,2538,17,...,joint,0.0,161,27,3,206,5,8,2538,0.0
3819041,632103,0,-1,0,eatout,Home,eatout,2584,2116,8,...,joint,0.0,236,21,2,134,12,6,2584,100.0
3819042,632103,0,-1,1,eatout,eatout,Home,2116,2584,8,...,joint,0.0,134,12,6,236,21,2,2584,100.0
3819171,634809,0,-1,0,othmaint,Home,othmaint,2595,257,15,...,joint,1.0,100,2,9,105,2,4,2595,100.0


In [12]:
test = ind_trip[ind_trip['hh_id'] == 1453060]
test.sort_values(['person_id', 'depart_hour'])

,hh_id,person_id,person_num,tour_id,stop_id,inbound,tour_purpose,orig_purpose,dest_purpose,orig_taz,...,parking_taz,depart_hour,trip_mode,tour_mode,tour_category,avAvailable,sampleRate,taxiWait,singleTNCWait,sharedTNCWait
125,1453060,3775459,1,0,-1,0,work_med,Home,work,1,...,0,7,7,7,MANDATORY,0,0.15,14.728372,9.301712,12.131115
126,1453060,3775459,1,0,-1,1,work_med,work,Home,2415,...,0,18,7,7,MANDATORY,0,0.15,7.296721,4.900870,6.700861
127,1453060,3775459,1,0,-1,0,shopping,Home,shopping,1,...,0,19,1,2,INDIVIDUAL_NON_MANDATORY,0,0.15,19.970312,12.661912,15.431287
128,1453060,3775459,1,0,-1,1,shopping,shopping,Home,522,...,0,20,1,2,INDIVIDUAL_NON_MANDATORY,0,0.15,5.093204,3.142239,5.373433
133,1453060,3775460,2,1,-1,0,othdiscr,Home,othdiscr,1,...,0,6,2,2,INDIVIDUAL_NON_MANDATORY,0,0.15,4.769279,2.968533,4.976103
134,1453060,3775460,2,1,-1,1,othdiscr,othdiscr,Home,525,...,0,6,1,2,INDIVIDUAL_NON_MANDATORY,0,0.15,2.992850,2.217732,4.306361
131,1453060,3775460,2,0,-1,0,shopping,Home,shopping,1,...,0,9,1,2,INDIVIDUAL_NON_MANDATORY,0,0.15,10.212216,6.419128,9.082714
129,1453060,3775460,2,0,-1,0,university,Home,university,1,...,0,17,6,6,MANDATORY,0,0.15,12.242913,7.713601,10.482464
132,1453060,3775460,2,0,-1,1,shopping,shopping,Home,214,...,0,17,2,2,INDIVIDUAL_NON_MANDATORY,0,0.15,5.296143,3.223740,5.461563
130,1453060,3775460,2,0,-1,1,university,university,Home,600,...,0,20,6,6,MANDATORY,0,0.15,4.278999,2.803244,4.997517


In [58]:
df = out_tripdata.copy()
df['new_dest_purp'] = df['dest_purpose']
df['new_orig_purp'] = df['orig_purpose']

In [59]:
df.loc[(df['tour_purpose'] == 'atwork_eat') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'eatout'
df.loc[(df['tour_purpose'] == 'atwork_eat') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'eatout'

df.loc[(df['tour_purpose'] == 'atwork_business') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'business'
df.loc[(df['tour_purpose'] == 'atwork_business') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'business'

df.loc[(df['tour_purpose'] == 'atwork_maint') & (df['dest_purpose'] == 'atwork'), 'new_dest_purp'] = 'othmaint'
df.loc[(df['tour_purpose'] == 'atwork_maint') & (df['orig_purpose'] == 'atwork'), 'new_orig_purp'] = 'othmaint'

In [16]:
# For last trip on Tour:
#    If stop destination = home:
#       If Tour Purpose = Work, then trip purpose = work
#       If Tour Purpose = School, then trip purpose = school
#       Else, trip purpose = trip origin purpose
#   If stop destination = work (i.e. work-based tour):
#       If Tour Purpose = Business, then trip purpose = business
#       Else, trip purpose = trip origin purpose
# For all other trips, trip purpose = stop destination purpose

In [51]:
#df['dest_purpose'].value_counts(), df['new_orig_purp'].value_counts(), df['new_dest_purp'].value_counts(), df['tour_purpose'].value_counts()

In [65]:
df['link21_tour_purp'] = df['tour_purpose'].map(link21_purp_mapping)
df['link21_orig_purp'] = df['new_orig_purp'].map(link21_purp_mapping)
df['link21_dest_purp'] = df['new_dest_purp'].map(link21_purp_mapping)

df['link21_trip_purp'] = df['link21_dest_purp']

In [66]:
print(df['link21_trip_purp'].value_counts())

df1 = df.loc[(df['link21_dest_purp'] == 'home')]
print(df1['link21_trip_purp'].value_counts())
conditions = [
    df1['link21_tour_purp'].eq('work'),
    df1['link21_tour_purp'].eq('school'),
    ~df1['link21_tour_purp'].isin(['work','school'])
]

choices = ['work', 'school', df1['link21_orig_purp']]
df1['link21_trip_purp'] = np.select(conditions, choices, default=0)
print(df1['link21_trip_purp'].value_counts())

df2 = df.loc[(df['link21_dest_purp'] != 'home')]
df2['link21_trip_purp'] = df2['link21_dest_purp']
print(df2['link21_trip_purp'].value_counts())

df = pd.concat([df1, df2], ignore_index=True)
print(df['link21_trip_purp'].value_counts())

home        1397549
work         654588
shopping     393883
social       373272
escort       321643
othmaint     251589
othdiscr     220144
school       210599
business      19679
Name: link21_trip_purp, dtype: int64
home    1397549
Name: link21_trip_purp, dtype: int64
work        463725
school      209109
shopping    207721
escort      142020
social      140961
othdiscr    132627
othmaint    101386
Name: link21_trip_purp, dtype: int64
work        654588
shopping    393883
social      373272
escort      321643
othmaint    251589
othdiscr    220144
school      210599
business     19679
Name: link21_trip_purp, dtype: int64
work        1118313
shopping     601604
social       514233
escort       463663
school       419708
othmaint     352975
othdiscr     352771
business      19679
Name: link21_trip_purp, dtype: int64


In [67]:
print(df1['link21_trip_purp'].value_counts())

work        463725
school      209109
shopping    207721
escort      142020
social      140961
othdiscr    132627
othmaint    101386
Name: link21_trip_purp, dtype: int64


In [69]:
print(df['link21_trip_purp'].value_counts())
df1 = df.loc[df['dest_purpose'] == 'atwork']
print(df1['link21_trip_purp'].value_counts())

conditions = [
    df1['link21_tour_purp'].eq('business'),
    ~df1['link21_tour_purp'].eq('business')
]

choices = ['business', df1['link21_orig_purp']]
df1['link21_trip_purp'] = np.select(conditions, choices, default=0)
print(df1['link21_trip_purp'].value_counts())


df2 = df.loc[(df['dest_purpose'] != 'atwork')]

df = pd.concat([df1, df2], ignore_index=True)
print(df['link21_trip_purp'].value_counts())

work        1118313
shopping     601604
social       514233
escort       463663
school       419708
othmaint     352975
othdiscr     352771
business      19679
Name: link21_trip_purp, dtype: int64
social      92062
business    19679
othmaint    14831
Name: link21_trip_purp, dtype: int64
work        99299
business    19679
othmaint     2347
social       2335
shopping     1575
escort       1199
othdiscr      138
Name: link21_trip_purp, dtype: int64
work        1217612
shopping     603179
escort       464862
social       424506
school       419708
othdiscr     352909
othmaint     340491
business      19679
Name: link21_trip_purp, dtype: int64


In [70]:
#out_tripdata['trips'] = 1

In [73]:
df.to_parquet(r"C:\VY-Projects\Link21\BaseYear2015\trips.parquet")

## Dashboard Data

In [26]:
df = out_tripdata.groupby(['orig_taz', 'dest_taz', 'orig_rdm_zones', 'dest_rdm_zones', 'orig_super_dist', 'dest_super_dist',
                           'orig_county', 'dest_county', 'transbay_od', 'trip_purpose', 'trip_mode', 'depart_hour', 'pp_share'])['trips'].sum().reset_index()

In [27]:
df[1:10]

,orig_taz,dest_taz,orig_rdm_zones,dest_rdm_zones,orig_super_dist,dest_super_dist,orig_county,dest_county,transbay_od,trip_purpose,trip_mode,depart_hour,pp_share,trips
1,1,1,75,75,26,26,4,4,0.0,othdiscr,2,21,0.0,1
2,1,1,75,75,26,26,4,4,0.0,othdiscr,4,11,0.0,1
3,1,1,75,75,26,26,4,4,0.0,othdiscr,4,15,0.0,1
4,1,1,75,75,26,26,4,4,0.0,othdiscr,4,18,0.0,3
5,1,1,75,75,26,26,4,4,0.0,othdiscr,4,19,0.0,1
6,1,1,75,75,26,26,4,4,0.0,social,4,17,0.0,2
7,1,1,75,75,26,26,4,4,0.0,work_med,4,8,0.0,1
8,1,1,75,75,26,26,4,4,0.0,work_med,4,18,0.0,1
9,1,3,75,48,26,16,4,4,0.0,othmaint,1,17,0.0,1


In [28]:
df['trip_purpose'].value_counts()

work_very high     507708
shopping           439435
othdiscr           353602
work_high          299212
school_grade       292704
escort_kids        260629
othmaint           256661
work_med           236340
atwork_eat         153366
eatout             152627
work_low           126548
social             111760
escort_no kids      78895
university          75133
school_high         54046
atwork_business     37473
atwork_maint        28964
Name: trip_purpose, dtype: int64

In [30]:
purpose = {'work_very high': 1,
'shopping': 2,
'othdiscr': 3,
'work_high' : 4,
'school_grade': 5,
'escort_kids': 6,
'othmaint': 7,
'work_med': 8,
'atwork_eat': 9,
'eatout': 10,
'work_low': 11,
'social': 12,
'escort_no kids': 13,
'university': 14,
'school_high': 15,
'atwork_business': 16,
'atwork_maint': 17}

In [32]:
df['trip_purpose'] = df['trip_purpose'].map(purpose)

In [33]:
df.shape

(3465103, 14)

In [35]:
df['trip_mode'].value_counts()

1    1517140
2     820691
3     497809
4     338116
6     166931
5      76723
7      20820
8      17444
9       9429
Name: trip_mode, dtype: int64

In [37]:
df['scenario'] = 'case1'

In [38]:
df.to_parquet(r'C:\Users\vyadav\OneDrive - Cambridge Systematics\CS-Projects\LINK21\data\trips.parquet')